In [1]:
import numpy as np
import matplotlib.pylab as plt
import random
from scipy.special import comb

import sys
sys.path.append('../src/')
sys.path.append('../scripts/')

In [2]:
import numpy as np
import networkx as nx
import random
from itertools import combinations

In [12]:
def regular_maximum_overlapped_simplicial_complex(n, k, seed=None):
    """
    -----------------------------------------------------------------------------------
    Thanks to Luca Gallo - https://scholar.google.com/citations?hl=it&user=sKiSU9AAAAAJ
    -----------------------------------------------------------------------------------
    
    Returns a regular simplicial complex of order 2 with maximum value of intra-order hyperedge overlap.

    Each node has exactly
    - 1-degree = k
    - 2-degree = (k-1)(k-2)/2
    We first fix a value of initial number of nodes n, the final structure will be will be composed of N = k*n nodes.
    
    Parameters
    ----------
    k : int
      The degree of each node.
    n : int
      The number of nodes for the starting k-regular random graph. 
      The value of $N \times k$ must be even.
      The numer of nodes of the simplicial complex will be k * n <-- 
    seed : int, random_state, or None (default)
        Indicator of random number generation state.
        
    Output
    ----------
    N : int
      The number of nodes in the simplicial complex.
    edges_list : ndarray
      Array composed by a list of tuples [(0,1),(2,3),...,etc] composing the set of 1-simplices
    triangles_list : ndarray
      Array composed by a list of tuples [(0,1,2),(2,3,4),...,etc] representing the set of 2-simplices
    """
    # Kim & Vu (2003) Generating Random Regular Graphs 
    # NOTE: asympt uniform if d << n^(1/3), which holds since we want sparser graphs
    M = nx.random_regular_graph(k,n,seed=seed) 
    
    graph_to_simplex = {ii:[ii*k+r for r in range(k)] for ii in range(n)}
    
    G = nx.disjoint_union_all([nx.complete_graph(k) for ii in range(n)]) # <- here `ii`
    
    triangle_list = []
    for clique in nx.enumerate_all_cliques(G):
        if len(clique)==3:
            triangle_list.append(clique)
        
    for edge in M.edges():
        m1 = edge[0]
        m2 = edge[1]
        n1 = random.choice(graph_to_simplex[m1])
        n2 = random.choice(graph_to_simplex[m2])
        
        G.add_edge(n1,n2)
        
        graph_to_simplex[m1].remove(n1)
        graph_to_simplex[m2].remove(n2)
        
    edges_list = np.array(G.edges())
    N = n*k
    return N, np.array(edges_list), np.array(triangle_list)

In [19]:
n = 200
k = 3
N = n * k
print(f"N = n * k = {N}")
print(f"N % 2 == 0: {N % 2 == 0}")

N = n * k = 600
N % 2 == 0: True


In [17]:
N_realized, np_edges, np_triangles = regular_maximum_overlapped_simplicial_complex(n, k)
N_realized, len(np_edges), len(np_triangles)

(1500, 2250, 500)

In [18]:
edges = set([(int(u), int(v)) for (u, v) in list(np_edges)])
triangles = set([(int(u), int(v), int(w)) for (u, v, w) in list(np_triangles)])
len(edges), len(triangles)

(2250, 500)